In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = sns.load_dataset('tips')

In [3]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [5]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [6]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [7]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [ ]:
#predict time on the basis of other features

In [ ]:
#dependent features is time

In [ ]:
#now convert time into numerical features and this can be done by using "Label Encoder"

In [8]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])

In [9]:
df['time'].unique()

array([0, 1])

In [10]:
#independent and dependent features
X=df.drop(labels=['time'],axis=1)
y=df.time

In [11]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [12]:
X['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [13]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [15]:
from sklearn.pipeline import Pipeline #to handle numerical and categorical features together
from sklearn.impute import SimpleImputer #to handle missing values
from sklearn.preprocessing import StandardScaler #to scale the features(feature scaling)
from sklearn.preprocessing import OneHotEncoder #to handle categorical features(convert categorical to numerical features)
from sklearn.compose import ColumnTransformer #to handle all the features together

In [16]:
#segregate the data
categorical_features = ['sex','smoker','day']
numerical_features = ['total_bill','tip','size']

In [17]:
#create a different column for numerical features to handle missing values
# Numerical Pipelines
num_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')), #missing values handle
        ('scaler',StandardScaler()) #feature scaling
    ]
)

#categorical pipeline
cat_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')), #missing values handle
        ('onehotencoder',OneHotEncoder()) #feature encoding[categorical features to numerical features]
    ]
)

In [18]:
#combining both the pipelines
#to do this we will use column transformer
preprocessor =ColumnTransformer(
    transformers=[
        ('num_pipeline',num_pipeline,numerical_features),
        ('cat_pipeline',cat_pipeline,categorical_features)
    ]
)

In [19]:
X_train = preprocessor.fit_transform(X_train)

In [20]:
X_test = preprocessor.transform(X_test)

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [38]:
##Model Training Automation
models = {
    'Random Forest':RandomForestClassifier(),

    'Logistic Regression':LogisticRegression(),

    'Decision Tree':DecisionTreeClassifier(),

}

In [39]:
from sklearn.metrics import accuracy_score

In [42]:
def evaluate_model(X_train,y_train,X_test,y_test,models):
  report = {}
  for i in range(len(models)):
    model = list(models.values())[i]
    #Train model
    model.fit(X_train,y_train)

    #Predict model(testing data)
    y_test_pred = model.predict(X_test)

    #Get accuracy for test data
    test_model_score = accuracy_score(y_test,y_test_pred)

    report[list(models.keys())[i]] = test_model_score


  return report

In [43]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.9753086419753086,
 'Logistic Regression': 0.9876543209876543,
 'Decision Tree': 0.9506172839506173}

In [44]:
classifier = RandomForestClassifier()

In [46]:
##Hyperparameter tuning
params={'max_depth':[3,5,10,None],
        'n_estimators':[100,200,300],
        'criterion':['gini','entropy']
        }

In [47]:
from sklearn.model_selection import RandomizedSearchCV

In [49]:
cv = RandomizedSearchCV(classifier,params,scoring='accuracy',cv=5,verbose =3)

In [50]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.939 total time=   0.7s
[CV 2/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.970 total time=   0.8s
[CV 3/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.909 total time=   0.7s
[CV 4/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.938 total time=   0.9s
[CV 5/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.906 total time=   1.1s
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=300;, score=0.939 total time=   0.9s
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=300;, score=0.970 total time=   1.0s
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=300;, score=0.909 total time=   0.5s
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=300;, score=0.938 total time=   0.5s
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=300;, score=0.906 tot

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [51]:
cv.best_params_

{'n_estimators': 200, 'max_depth': 3, 'criterion': 'entropy'}